# Import Library

In [1]:
import pandas as pd

import warnings
warnings.simplefilter("ignore")

C:\Users\Joshua Kan\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Joshua Kan\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\Joshua Kan\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


# Grab the data

In [2]:
# we will be using these columns
use_cols = ["Organization ID", "Organization Name", "Project Type", "address1", "address2", "city", "state", "zip"]

In [3]:
# read the data from HUD
df = pd.read_excel("https://www.huduser.gov/portal/sites/default/files/xls/2019-Housing-Inventory-County-RawFile.xlsx", usecols=use_cols)

In [4]:
df.head()

,Organization ID,Organization Name,Project Type,address1,address2,city,state,zip
0,495,Battered Women's Shelter,ES,NaN,NaN,NaN,NaN,NaN
1,495,Battered Women's Shelter,TH,NaN,NaN,NaN,NaN,NaN
2,495,Battered Women's Shelter,RRH,NaN,NaN,NaN,NaN,NaN
3,23705,Crisis Center of Russell County,ES,NaN,NaN,NaN,NaN,NaN
4,1943,Daybreak,ES,NaN,NaN,NaN,NaN,NaN


In [5]:
# all unique values for project type 
df["Project Type"].unique()

array(['ES', 'TH', 'RRH', 'PSH', 'OPH', 'SH'], dtype=object)

The values in **Project Type** column are : 
 * Emergency Shelter (ES)
 * Transitional Housing (TH)
 * Safe Haven (SH)
 * Permanent Supportive Housing (PSH)
 * Rapid Re-Housing (RRH)
 * Other PH (OPH)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25142 entries, 0 to 25141
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Organization ID    25142 non-null  int64  
 1   Organization Name  25142 non-null  object 
 2   Project Type       25142 non-null  object 
 3   address1           19920 non-null  object 
 4   address2           307 non-null    object 
 5   city               20187 non-null  object 
 6   state              21179 non-null  object 
 7   zip                23388 non-null  float64
dtypes: float64(1), int64(1), object(6)
memory usage: 1.5+ MB


There are no missing values on **Organization ID**, **Organization Name**, and **Project Type** columns. 

There are more than 5000 missing values on **address1**, and more than 20,000 missing values on **address2**. This make sense because **address2** column is for the secondary address (e.g. apt number, PO Box, etc). 

There are also some missing values on **city** and **state** columns. We would drop these 2 columns and get more accurate values later on by joining to the other data. 

There are also around 1,500 missing values on **zip** column.

We assume that **Organization ID** and **Organization Name** are bijective. Let's make sure that this is true

In [7]:
df = df.drop(["city", "state"], axis = 1)

In [235]:
df

,Organization ID,Organization Name,Project Type,address1,address2,zip
0,495,Battered Women's Shelter,ES,NaN,NaN,NaN
1,495,Battered Women's Shelter,TH,NaN,NaN,NaN
2,495,Battered Women's Shelter,RRH,NaN,NaN,NaN
3,23705,Crisis Center of Russell County,ES,NaN,NaN,NaN
4,1943,Daybreak,ES,NaN,NaN,NaN
...,...,...,...,...,...,...
25137,38857,Wyoming Rescue Mission,ES,230 N PARK ST,NaN,82601.0
25138,41520,Youth Alternative Home Association,ES,905 North Gurley Ave,NaN,82716.0
25139,18486,"Youth Emergency Services, Inc.",ES,905 N GURLEY AVE,NaN,82716.0
25140,18486,"Youth Emergency Services, Inc.",ES,905 N Gurley Ave,NaN,82716.0


Nice and clean

# For later analysis, we want to make an assumption that if 2 rows have the same **Oranization ID/Name** and **zip**, then they have the same address. Let's proof this by looking at the data

# Feature engineering

For **Project Type** column, we will use one-hot representation. The result is the following:

In [157]:
# one hot encode the "Project Type column"
df2 = pd.concat([df, pd.get_dummies(df["Project Type"])], axis = 1).drop("Project Type", axis = 1)

In [158]:
df2.head()

,Organization ID,Organization Name,address1,address2,zip,ES,OPH,PSH,RRH,SH,TH
0,495,Battered Women's Shelter,NaN,NaN,NaN,1,0,0,0,0,0
1,495,Battered Women's Shelter,NaN,NaN,NaN,0,0,0,0,0,1
2,495,Battered Women's Shelter,NaN,NaN,NaN,0,0,0,1,0,0
3,23705,Crisis Center of Russell County,NaN,NaN,NaN,1,0,0,0,0,0
4,1943,Daybreak,NaN,NaN,NaN,1,0,0,0,0,0


The value ES, OPH, PSH, RRH, SH, and TH are now a one hot columns. We could also see that the first 3 rows have the same **Organization Name** and **ID**. It's because our data treat the different **Project Type** as a distinct observation. Later on, we would group our observation by **ID** and **Name**, then we would sum all of the one-hot columns. The result will be one organization for each row with several project types. 

<br>

In short, we assume that one zip code only corresponds to unique address for a set of **Organization ID** and **Organization Name**

<br>

However, there are some problem that occurs, there are an organization with multiple addresses. 

In [159]:
# an example of an organization with multiple addresses
df2[df2["Organization ID"] == 7]

,Organization ID,Organization Name,address1,address2,zip,ES,OPH,PSH,RRH,SH,TH
159,7,ABCCM,NaN,NaN,28801.0,0,0,0,1,0,0
15159,7,ABCCM,30 CUMBERLAND AVE,NaN,28801.0,1,0,0,0,0,0
15160,7,ABCCM,30 CUMBERLAND AVE,NaN,28801.0,0,0,0,0,0,1
15161,7,ABCCM,1329 TUNNEL RD,NaN,28805.0,1,0,0,0,0,0
15162,7,ABCCM,1329 TUNNEL RD,NaN,28805.0,0,1,0,0,0,0
15163,7,ABCCM,1329 TUNNEL RD,NaN,28805.0,0,0,0,0,0,1
15164,7,ABCCM,1329 TUNNEL RD,NaN,28805.0,0,0,0,0,0,1
15545,7,ABCCM,1329 TUNNEL RD,NaN,28805.0,1,0,0,0,0,0


It's shown that there are some organization that have multiple housing location. Therefore we would also group by **zip** or **address** after cleaning the data.

# Data Cleaning

### Combine and Impute the address and city

In [160]:
# manipulate the string values
df2["address1"] = df2["address1"].str.upper()
df2["address2"] = df2["address2"].str.upper()
df2["zip"] = df2["zip"].fillna("").apply(lambda x: str(x)[:-2]) # we wanted to get rid of floating representation
df2["address1"] = df2["address1"].fillna("")
df2["address2"] = df2["address2"].fillna("")

Here we fill NaN value by an empty string. The reason is because we want to use boolean operation for those columns later. We will then combine the **address1** and **address2** columns into one column called **address**.

In [161]:
# combine the address
df2["address"] = df2["address1"] + df2["address2"]

# drop address1 and address2
df2 = df2.drop(["address1", "address2"], axis = 1)

In [162]:
df2.shape

(25142, 10)

In [163]:
df2[df2["Organization ID"] == 7]

,Organization ID,Organization Name,zip,ES,OPH,PSH,RRH,SH,TH,address
159,7,ABCCM,28801,0,0,0,1,0,0,
15159,7,ABCCM,28801,1,0,0,0,0,0,30 CUMBERLAND AVE
15160,7,ABCCM,28801,0,0,0,0,0,1,30 CUMBERLAND AVE
15161,7,ABCCM,28805,1,0,0,0,0,0,1329 TUNNEL RD
15162,7,ABCCM,28805,0,1,0,0,0,0,1329 TUNNEL RD
15163,7,ABCCM,28805,0,0,0,0,0,1,1329 TUNNEL RD
15164,7,ABCCM,28805,0,0,0,0,0,1,1329 TUNNEL RD
15545,7,ABCCM,28805,1,0,0,0,0,0,1329 TUNNEL RD


Now we verify our assumption on zip, address, and Organization ID & Name.

One organization with same address is super less likely to have different zip:
 * we can impute zip

In [254]:
def impute_df(df, column_name):
    
    if column_name == "address":
        other_column = "zip"
    elif column_name == "zip":
        other_column = "address"
    else:
        return None
    
    df_result = df.copy()
    df_result = df_result.sort_values(by = column_name, ascending = False).reset_index(drop = True)
    df_result = df_result.sort_values(by = ["Organization ID",
                                       other_column]).reset_index(drop = True)
    
    # iterate over all observation
    for i in range(1, df_result.shape[0]):
        
        # if there are missing values on column_name
        if df_result[column_name][i] == "":
            
            # if the organization id, and zip are the same with the previous one
            # we assume that rows with same organization and zip will have the same address
            if df_result["Organization ID"][i] == df_result["Organization ID"][i-1] and df_result[other_column][i] == df_result[other_column][i-1] and df_result[other_column][i]!="":
                
                df_result[column_name][i] = df_result[column_name][i-1]
    return df_result

In [255]:
df_verify = impute_df(df2, "zip")

In [256]:
df_verify[df_verify["zip"]==""].shape # missing value for zip after imputing zip

(1754, 10)

In [257]:
df2[df2["zip"]==""].shape # missing value for zip before imputing zip

(1754, 10)

if zip is missing, then address is also missing because we don't have result when imputing zip

So we can just drop missing zip from df2, so we don't have missing zip anymore.

In [258]:
df3 = df2[df2["zip"]!= ""]

In [259]:
df3.shape

(23388, 10)

In [260]:
df4 = impute_df(df3, "address")

In [261]:
df4[df4["address"]==""].shape

(2397, 10)

In [262]:
df3[df3["address"]==""].shape

(3480, 10)

we impute around 1000 missing address

we want to drop df3 that has missing values on zip and address because it doesn't has location information, it tooks a long time to search google for the address one by one

In [268]:
df4

,Organization ID,Organization Name,zip,ES,OPH,PSH,RRH,SH,TH,address
0,1,12 & 12,74119,0,0,0,0,0,1,1214 S. BALTIMORE AVE.
1,1,12 & 12,74135,0,0,0,0,0,1,6333 E SKELLY DR
2,1,12 & 12,74135,0,0,0,0,0,1,6333 E SKELLY DR
3,3,24 Hour Oakland Parent / Teacher Children's Ce...,94601,1,0,0,0,0,0,4700 INTERNATIONAL BLVD
4,6,Abby's House,1609,1,0,0,0,0,0,23 CROWN ST
...,...,...,...,...,...,...,...,...,...,...
23383,43460,FEMA,32401,1,0,0,0,0,0,
23384,43461,Soldier On,8619,0,0,0,1,0,0,
23385,43462,The Veterans Multi-Service Center - Burlington,8332,0,0,0,1,0,0,
23386,43463,Federal Emergency Management Agency,95482,0,0,0,0,0,1,


In [270]:
df4[df4["address"]==""]

,Organization ID,Organization Name,zip,ES,OPH,PSH,RRH,SH,TH,address
13,10,Abused Women's Aid in Crisis (AWAIC),99501,0,0,0,0,0,1,
14,10,Abused Women's Aid in Crisis (AWAIC),99501,1,0,0,0,0,0,
15,10,Abused Women's Aid in Crisis (AWAIC),99504,0,0,0,0,0,1,
25,19,Action in Community Through Service (ACTS),20136,0,0,0,1,0,0,
44,23,Advocates for Mentally Ill Housing,95661,0,0,0,1,0,0,
...,...,...,...,...,...,...,...,...,...,...
23383,43460,FEMA,32401,1,0,0,0,0,0,
23384,43461,Soldier On,8619,0,0,0,1,0,0,
23385,43462,The Veterans Multi-Service Center - Burlington,8332,0,0,0,1,0,0,
23386,43463,Federal Emergency Management Agency,95482,0,0,0,0,0,1,


In [271]:
df5 = df4[df4["address"]!=""]

In [275]:
df5.shape

(20991, 10)

In [278]:
df6 = df5.drop_duplicates(df5.columns)

In [279]:
df6.shape

(16802, 10)

In [282]:
df6[df6["Organization ID"]==7]

,Organization ID,Organization Name,zip,ES,OPH,PSH,RRH,SH,TH,address
5,7,ABCCM,28801,1,0,0,0,0,0,30 CUMBERLAND AVE
6,7,ABCCM,28801,0,0,0,0,0,1,30 CUMBERLAND AVE
7,7,ABCCM,28801,0,0,0,1,0,0,30 CUMBERLAND AVE
8,7,ABCCM,28805,1,0,0,0,0,0,1329 TUNNEL RD
9,7,ABCCM,28805,0,0,0,0,0,1,1329 TUNNEL RD
10,7,ABCCM,28805,0,1,0,0,0,0,1329 TUNNEL RD


In [291]:
df7 = df6.groupby(["Organization ID", "address"]).sum().reset_index()
df7

,Organization ID,address,ES,OPH,PSH,RRH,SH,TH
0,1,1214 S. BALTIMORE AVE.,0,0,0,0,0,1
1,1,6333 E SKELLY DR,0,0,0,0,0,1
2,3,4700 INTERNATIONAL BLVD,1,0,0,0,0,0
3,6,23 CROWN ST,1,0,0,0,0,0
4,7,1329 TUNNEL RD,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...
14234,43450,805 LINCOLN STREET,0,0,0,0,0,1
14235,43451,979 BRANCH AVE,1,0,0,0,0,0
14236,43452,435 N SINGLETON AVE.,1,0,0,0,0,0
14237,43454,711 PARK AVE WEST,1,0,0,0,0,0


In [292]:
df7[df7["Organization ID"]==7]

,Organization ID,address,ES,OPH,PSH,RRH,SH,TH
4,7,1329 TUNNEL RD,1,1,0,0,0,1
5,7,30 CUMBERLAND AVE,1,0,0,1,0,1


In [293]:
id_dict = {}
for id_, name_ in zip(df6["Organization ID"].values, df6["Organization Name"].values):
    id_dict[id_] = name_

In [295]:
zip_dict = {}

for id_, zip_, address_ in zip(df6["Organization ID"].values, df6["zip"].values, df6["address"].values):
    
    zip_dict[(id_, address_)] = zip_

In [311]:
df7["Organization Name"] = df7["Organization ID"].apply(lambda x: id_dict[x])

In [317]:
df7["zip"] = df7.apply(lambda x: zip_dict[(x["Organization ID"], x["address"])], axis = 1)

In [368]:
project_type = np.array(["ES", "OPH", "PSH", "RRH", "SH", "TH"])

df7["summary"] = df7.apply(lambda s: ", ".join(project_type[s[project_type] > 0]), axis = 1)

In [369]:
df7.head()

,Organization ID,address,ES,OPH,PSH,RRH,SH,TH,Organization Name,zip,summary
0,1,1214 S. BALTIMORE AVE.,0,0,0,0,0,1,12 & 12,74119,TH
1,1,6333 E SKELLY DR,0,0,0,0,0,1,12 & 12,74135,TH
2,3,4700 INTERNATIONAL BLVD,1,0,0,0,0,0,24 Hour Oakland Parent / Teacher Children's Ce...,94601,ES
3,6,23 CROWN ST,1,0,0,0,0,0,Abby's House,1609,ES
4,7,1329 TUNNEL RD,1,1,0,0,0,1,ABCCM,28805,"ES, OPH, TH"


In [371]:
df7.drop(project_type, axis = 1, inplace = True)

In [390]:
df7.shape

(14239, 5)

### Impute the rest of the columns (city, state) using other csv file

In [376]:
df_zip = pd.read_csv("USZipCodes202003.csv")

In [377]:
df_zip["Zip Code"] = df_zip["Zip Code"].apply(lambda x: str(x))

In [378]:
df_zip.head()

,Zip Code,City,County,State,CountyFIPS,StateFIPS,TimeZone,DayLightSavings,ZipLatitude,ZipLongitude
0,501,HOLTSVILLE,SUFFOLK,NY,103.0,36.0,5.0,Y,40.815400,-73.045600
1,544,HOLTSVILLE,SUFFOLK,NY,103.0,36.0,5.0,Y,40.815400,-73.045600
2,601,ADJUNTAS,ADJUNTAS,PR,1.0,72.0,4.0,N,18.196747,-66.736735
3,602,AGUADA,AGUADA,PR,3.0,72.0,4.0,N,18.352927,-67.177532
4,603,AGUADILLA,AGUADILLA,PR,5.0,72.0,4.0,N,18.458585,-67.129867


In [395]:
df_zip[df_zip["Zip Code"]=="2904"]

,Zip Code,City,County,State,CountyFIPS,StateFIPS,TimeZone,DayLightSavings,ZipLatitude,ZipLongitude
1345,2904,N PROVIDENCE,PROVIDENCE,RI,7.0,44.0,5.0,Y,41.854638,-71.437492
1346,2904,NORTH PROVIDENCE,PROVIDENCE,RI,7.0,44.0,5.0,Y,41.854638,-71.437492
1347,2904,PROVIDENCE,PROVIDENCE,RI,7.0,44.0,5.0,Y,41.854638,-71.437492


In [407]:
df_final = df7.merge(df_zip, right_on="Zip Code", left_on="zip", how = "left")[["Organization ID", "Organization Name", "address",
                                                                                "State", "Zip Code", "summary"]]

In [409]:
df_final.drop_duplicates()

,Organization ID,Organization Name,address,State,Zip Code,summary
0,1,12 & 12,1214 S. BALTIMORE AVE.,OK,74119,TH
1,1,12 & 12,6333 E SKELLY DR,OK,74135,TH
2,3,24 Hour Oakland Parent / Teacher Children's Ce...,4700 INTERNATIONAL BLVD,CA,94601,ES
3,6,Abby's House,23 CROWN ST,MA,1609,ES
4,7,ABCCM,1329 TUNNEL RD,NC,28805,"ES, OPH, TH"
...,...,...,...,...,...,...
24802,43450,Youth Advocates of Sitka,805 LINCOLN STREET,AK,99835,TH
24803,43451,Holy Family Home for Women and Children (Non-P...,979 BRANCH AVE,RI,2904,ES
24806,43452,Disabled Amer. Vet. Chapter 109,435 N SINGLETON AVE.,FL,32796,ES
24807,43454,MDHI,711 PARK AVE WEST,CO,80205,ES


In [433]:
s = df7["Organization ID"].sort_values().value_counts()

In [437]:
x = df_final.drop_duplicates()["Organization ID"].sort_values().value_counts()

In [442]:
df_checker = pd.concat([s, x], axis = 1)

In [449]:
index_duplicated = df_checker[df_checker.iloc[:,1]>df_checker.iloc[:,0]].index

In [464]:
df_final = df_final.drop_duplicates()

In [466]:
df_final.to_csv("HUD_clean.csv", index = False)

In [461]:
len(index_duplicated)

21

In [463]:
df_final_2[df_final_2["Organization ID"]==index_duplicated[3]]

,Organization ID,Organization Name,address,State,Zip Code,summary
11675,9031,Loeb House Inc.,10 - 24 FRED HECHT DRIVE,NY,10977,PSH
11679,9031,Loeb House Inc.,10-24 FRED HECHT DRIVE,NY,10977,ES
11683,9031,Loeb House Inc.,2 PATH OF HEROES WAY,NJ,10983,PSH
11684,9031,Loeb House Inc.,2 PATH OF HEROES WAY,NY,10983,PSH
11685,9031,Loeb House Inc.,96 OLD ORANGEBURG RD,NY,10962,PSH


In [440]:
x

2322     327
42678    184
11962     45
6154      36
2324      35
        ... 
348        1
4446       1
6500       1
8547       1
32768      1
Name: Organization ID, Length: 7242, dtype: int64

In [396]:
s = df_final["Zip Code"].value_counts()

In [398]:
s[s > 2]

95076    147
2124     120
33040    119
18702    117
55418    110
        ... 
1109       3
30331      3
2364       3
96746      3
53105      3
Name: Zip Code, Length: 2485, dtype: int64

In [405]:
df_zip[df_zip["Zip Code"]=="95076"]

,Zip Code,City,County,State,CountyFIPS,StateFIPS,TimeZone,DayLightSavings,ZipLatitude,ZipLongitude
64861,95076,LA SELVA BEACH,MONTEREY,CA,53.0,6.0,8.0,Y,36.942773,-121.724031
64862,95076,ROYAL OAKS,SANTA CRUZ,CA,87.0,6.0,8.0,Y,36.942773,-121.724031
64863,95076,CORRALITOS,MONTEREY,CA,53.0,6.0,8.0,Y,36.942773,-121.724031
64864,95076,ROYAL OAKS,MONTEREY,CA,53.0,6.0,8.0,Y,36.942773,-121.724031
64865,95076,ROYAL OAKS,SANTA CLARA,CA,85.0,6.0,8.0,Y,36.942773,-121.724031
64866,95076,PAJARO,SANTA CLARA,CA,85.0,6.0,8.0,Y,36.942773,-121.724031
64867,95076,PAJARO,SANTA CRUZ,CA,87.0,6.0,8.0,Y,36.942773,-121.724031
64868,95076,CORRALITOS,SANTA CLARA,CA,85.0,6.0,8.0,Y,36.942773,-121.724031
64869,95076,CORRALITOS,SANTA CRUZ,CA,87.0,6.0,8.0,Y,36.942773,-121.724031
64870,95076,LA SELVA BEACH,SANTA CLARA,CA,85.0,6.0,8.0,Y,36.942773,-121.724031


In [404]:
df_final[df_final["Zip Code"] == "95076"][df_final[df_final["Zip Code"] == "95076"]["Organization ID"]==2510]

,Organization ID,Organization Name,address,City,State,Zip Code,summary
3933,2510,Families in Transition,406 MAIN ST STE 207,LA SELVA BEACH,CA,95076,RRH
3934,2510,Families in Transition,406 MAIN ST STE 207,ROYAL OAKS,CA,95076,RRH
3935,2510,Families in Transition,406 MAIN ST STE 207,CORRALITOS,CA,95076,RRH
3936,2510,Families in Transition,406 MAIN ST STE 207,ROYAL OAKS,CA,95076,RRH
3937,2510,Families in Transition,406 MAIN ST STE 207,ROYAL OAKS,CA,95076,RRH
3938,2510,Families in Transition,406 MAIN ST STE 207,PAJARO,CA,95076,RRH
3939,2510,Families in Transition,406 MAIN ST STE 207,PAJARO,CA,95076,RRH
3940,2510,Families in Transition,406 MAIN ST STE 207,CORRALITOS,CA,95076,RRH
3941,2510,Families in Transition,406 MAIN ST STE 207,CORRALITOS,CA,95076,RRH
3942,2510,Families in Transition,406 MAIN ST STE 207,LA SELVA BEACH,CA,95076,RRH
